## **04. 언론사**

In [3]:
import requests 
from bs4 import BeautifulSoup
import time
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
import webdriver_manager
from webdriver_manager.chrome import ChromeDriverManager
import pprint
import re
import pandas as pd
import urllib3
urllib3.disable_warnings()

In [4]:
service = ChromeService(executable_path=ChromeDriverManager().install())
options = Options()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36")
driver = webdriver.Chrome(service=service, options=options)

press = {'한겨레': '028', '경향신문': '032', '한국일보': '469', '동아일보': '020', '조선일보': '023', 
         '중앙일보': '025', '국민일보': '005', '문화일보': '021', '서울신문': '081', '세계일보': '022'} 

press_information = {}
    
for press_name, press_id in press.items():
    url = f"https://media.naver.com/press/{press_id}"
    driver.get(url)
    element = driver.find_element(By.CSS_SELECTOR, "body > div.press_wrap > div > section.press_content > header > div.press_hd_main > div")
    press_info = {}

    press_info['name'] = press_name
    
    subscriber = element.find_element(By.CSS_SELECTOR, "div.press_hd_info > div.press_hd_subscribe > span > em").text
    press_info['subscribers'] = int(re.sub(r'[^0-9]', '', subscriber))
        
    try:
        channels = element.find_element(By.CSS_SELECTOR, "div.press_profile > div > div > a > div.prd_left > div")
        channels = re.findall("\d+", channels.text)
        press_info['channel'] = int(channels[0])
        press_info['channels_subscribers'] = int(channels[1])
    except:
        press_info['channel'] = -1
        press_info['channels_subscribers'] = -1
        
    press_information[press_id] = press_info

driver.close()
driver.quit()

In [5]:
press_information['032']

{'name': '경향신문', 'subscribers': 400, 'channel': 2, 'channels_subscribers': 8}

In [6]:
press_information = pd.DataFrame(press_information).T.reset_index()
press_information.columns = ['press_id', 'press_name', 'naver_subscribers', 'channels', 'channels_subscribers']
press_information.head()

,press_id,press_name,naver_subscribers,channels,channels_subscribers
0,028,한겨레,400,1,57
1,032,경향신문,400,2,8
2,469,한국일보,400,4,25
3,020,동아일보,400,-1,-1
4,023,조선일보,500,-1,-1


In [7]:
press_information.to_csv("press.csv", index=False)